# Dask

Dask baut auf bewährten Modulen auf und erweitert diese um Möglichkeiten zur massiven Parallelisierung. So können mehrere NumPy Arrays oder Pandas Dataframes in entsprechenden Dask-Objekten zusammengefasst und für parallele Operationen bereitgestellt werden. Die Dask Objekte stellen große Teile der bakannten API (identisch zu NumPy Arrays oder Pandas Dataframes) bereit.

![image](images/Dask_Scale.svg)

## Dask Dashboard

Eine Übersicht über die von Dask gestarteten Parallelen Vorgänge und deren Auslastung kann über das Dask Dashboard eingesehen werden. Das Client Objekt aus dem dask.distributed Modul ermöglicht das Starten eines Dask Dashboards. Wird das initialisierte Client Objekt ausgegeben, so enthält die Ausgabe eine URL unter der das gestartete Dashboard abgerufen werden kann.

Wenn Jupter in Version 3.0 installiert ist oder zusätzlich Node.js (Version >= 12.0.0) und npm installiert sind, kann alternativ zur manuellen Nutzung auch das dask-labextensions Plugin in Jupyter installiert werden. Dies sorgt für eine Integration des Dask Dashboards in die Jupyter Oberfläche. Am linken Rand ist dann eine neue Schaltfläche "Dask" vorhanden. Über dies kann das Dask Dashboard erreicht werden, ohne dass hierfür eine separate URL aufgerufen werden muss.

In [9]:
from dask.distributed import Client
client = Client(processes=False, threads_per_worker=4,
                n_workers=1, memory_limit='2GB')
client

/home/julian/jupyterlab/lib/python3.8/site-packages/distributed/node.py:160: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 36895 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://94.16.122.16:36895/status,
Dashboard: http://94.16.122.16:36895/status,Workers: 1
Total threads: 4,Total memory: 1.86 GiB
Status: running,Using processes: False
Comm: inproc://94.16.122.16/286813/17,Workers: 1
Dashboard: http://94.16.122.16:36895/status,Total threads: 4
Started: Just now,Total memory: 1.86 GiB
Comm: inproc://94.16.122.16/286813/20,Total threads: 4
Dashboard: http://94.16.122.16:40727/status,Memory: 1.86 GiB
Nanny: None,


In [10]:
client.close()

Aktuell funktioniert das dask-labextensions Plugin am bwUniCluster noch nicht. Alternativ kann wie oben beschrieben direkt die URL des Dashboards genutzt werden. Hierfür muss über ssh der Port aus der URL aus dem Cluster nach außen weitergeleitet werden. Dies kann mit folgendem Befehl lokal am genutzten Rechner in einer Konsole durchgeführt werden. Der Port und die IP des Jupyter-Compute-Node können dabei der Dashboard-URL entnommen werden.

```bash
ssh -N -L <Port>:<Jupyter-Compute-Node>:<Port> <Hochschulkürzel>_<User-ID>@bwunicluster.scc.kit.edu
```

Nach Ausführen des ssh-Port-Forwardings kann am lokalen Rechner das Dask-Dashboard unter

```bash
http://localhost:<Port>/status
```

aufgerufen werden.

## Dask Array

Dask Array koordiniert mehrere NumPy Arrays und verteilt diese auf die zur Verfügung stehenden Ressourcen. So können Operationen verteilt auf mehrere Threads, Prozesse oder gar Nodes ausgeführt werden. Welche Operationen dabei möglich sind (welche Teile der NumPy Array API auch von Dask Array angeboten werden) kann der Dokumentation entnommen werden: https://docs.dask.org/en/latest/array-api.html.

Weitere Beispiele zu Dask Array: https://mybinder.org/v2/gh/dask/dask-examples/main?urlpath=lab/tree/array.ipynb

## Dask und SLURM

Um Dask in Kombination mit SLURM (dem Job-Scheduler des bwUniClusters) nutzen zu können, wird entweder die Klasse SLURMCluster aus dem Modul dask_jobqueue oder das Programm dask-mpi benötigt.

Für die nachfolgenden Übungen wird dask-mpi benötigt. dask_jobqueue ist daher nur grundlegend beschrieben. Eine Umgebung mit allen für die Übungen benötigten Modulen wird nur in der Beschreibung von dask-mpi erstellt.

### dask_jobqueue

WICHTIG: dask_jobqueue setzt eine eins zu eins Beziehung zwischen Job und Node vorraus. D.h. pro Job wird genau ein Node reserviert und genutzt. Werden mehrere Nodes benötigt, so sieht dask_jobqueue vor, dass entsprechend viele Jobs abgesetzt werden. Dies führt auf dem bwUniCluster zu zwei grundlegenden Problemen. Zum einen werden mehrere Jobs unabhängig voneinander gescheduled. Jeder Job hat daher einen eigenen Startzeitpunkt. Es stehen also nur mit viel Glück alle benötigten Ressourcen zeitgleich zur Verfügung. Zum anderen sehen alle multiple-Queues vor, dass pro Job mindestens zwei Nodes reserviert werden. Diese Queues sind explizit für Anwendungsfälle gedacht in denen mehrere Nodes zur gleichen Zeit benötigt werden. Werden diese Queues zusammen mit dask_jobqueue genutzt, so setzt dask_jobqueue für jeden benötigten Node einen eigenen Job ab, der jeweils zwei Nodes resserviert, von denen dask_jobqueue aber dann nur einen nutzt.

Fazit:
- um mehrere Nodes zeitgleich zur Verfügung zu haben, sieht bwUniCluster einen Job in einer multiple-Queue mit mehreren Nodes pro Job vor
- dask_jobqueue setzt vorraus, dass pro Job nur ein Node genutzt wird
- anstelle von dask_jobqueue sollte dask-mpi genutzt werden


Damit dask_jobqueue zur Verfügung steht muss im jeweiligen Environment sowohl dask als auch dask_jobqueue installiert sein:

```bash
python3 -m pip install dask_jobqueue dask
```

Ist der IPython-Kernel aus einem entsprechend erweiterten Environment im Jupyter registriert, so kann dieser beim Start eines neuen Notebooks ausgewählt werden. Anschließend kann die SLURMCluster Klasse im Notebook importiert und zum Erstellen einer SLURM-Job-Konfiguration genutzt werden.

Welche queues für eine solche Konfiguration am bwUniCluster zur Verfügung stehen und welche Eigenschaften diese Haben kann der Dokumentation unter

https://wiki.bwhpc.de/e/BwUniCluster_2.0_Batch_Queues

entnommen werden.

In [2]:
from dask_jobqueue import SLURMCluster
cluster = SLURMCluster(
    queue='multiple', # queue multiple ermöglicht eine Reservierung von mehreren Nodes pro Job (min. 2 Nodes pro Job)
    cores=40, # ein Node der queue multiple besitzt 40 cores => für 80 cores werden zwei Nodes angefordert
    memory="90GB", # maximal verfügbarer Speicher pro Node in queue multiple
    local_directory='/tmp', # Daten sollen lokal im Node und nicht über Netzwerk ins zentrale Filesystem geschrieben werden
    walltime='00:30:00', # Nodes sollen eine halbe Stunde reserviert werden
    interface='ib0' # für die Netzwerkkommunikation im Cluster wollen wir schnelles Infiniband nutzen
)

Der eigentliche Job wird dann auf Basis der Konfiguration mittels der Methode scale gestartet:

In [3]:
cluster.scale(jobs=1) # beim Start der Konfiguration können auch mehrere Jobs gleichzeitig gestartet werden (hierdurch ist das Reservieren mehrerer Nodes möglich)

In [4]:
from dask.distributed import Client
client = Client(cluster)
client # enthält die Informationen über den gestarteten Cluster

<Client: 'tcp://172.26.21.158:44599' processes=10 threads=80, memory=83.80 GiB>

### dask-mpi

Die Anwendung dask-mpi ermöglicht das Starten eines dask-Clusters über MPI. Hierdurch ist es möglich, mehrere Nodes mit nur einem Job zu reservieren. Dadurch können auch Queues genutzt werden, die mehr als einen Node pro Job voraussetzen. Zudem ist sichergestellt, dass alle benötigten Nodes zum gleichen Zeitpunkt verfügbar sind (da sie über den gleichen Job angefordert wurden).

Um dask-mpi mit allen für die Übungen benötigten Modulen verfügbar zu machen, muss zunächst ein entsprechendes Environment erstellt werden. Hierfür nutzen wir Miniconda.

Miniconda bietet die Möglichkeit ein Environment über das Laden von fertigen Binaries zu erstellen. Im Gegensatz zu pip install reduziert dies die nötigen Abhängigkeiten, da bei pip install gegebenenfalls Software gebaut wird (z.B. mpi für dask-mpi) und hierfür alle zum Bauen/Kompilieren notwendigen Komponenten in der jeweils passenden Form vorhanden sein müssen.

Zunächst benötigen wir eine aktuelle Version von Miniconda (der folgende Befehl muss wie alle nachfolgenden Befehle im Terminal File->New->Terminal ausgeführt werden):

```bash
wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
```

Anschließend muss das geladene sh-Script mit den zum Ausführen nötigen Rechten ausgestattet und dann ausgeführt werden:

```bash
chmod +x Miniconda3-latest-Linux-x86_64.sh
```

```bash
./Miniconda3-latest-Linux-x86_64.sh
```

Während der Ausführung des Scripts kommt zunächst die Abfrage "Please, press ENTER to continue". Diese bitte mit der Enter-Taste bestätigen.

Anschließend bitte den Lizenzvereinbarungen zustimmen. Mittels der Leertaste kann zum Ende der Vereinbarung gesprungen werden. Dannach erscheint die Abfrage "Please answer 'yes' or 'no':". Durch die Eingabe von "yes" und Betätigen der Enter-Taste kann fortgefahren werden.

Bei der Abfrage "Press ENTER to confirm the location" benötigen wir die Standardeinstellung. Dementsprechend bitte einfach mit der Enter-Taste bestätigen.

Die Abfrage "Do you wish the installer to initialize Miniconda3 by running conda init? \[yes|no\]" bitte mit "yes" bestätigen.

Nachdem die Installation von Miniconda abgeschlossen ist, ist das bei der Installation erstellte base-Environment in die bashrc-Datei eingetragen. Dies sorgt dafür, dass dieses Environment beim Starten eines Terminals automatisch aktiviert wird. Wir empfehlen diesen Automatismus über den folgenden Befehl zu deaktivieren:

```bash
conda config --set auto_activate_base false
```

Das für die folgenden Übungen benötigte Environment wird mit dem folgenden Befehl erstellt:

```bash
conda create -n python_workshop_env python=3
```

Die Abfrage "Proceed (\[y\]/n)?" bitte mit "y" bestätigen.

Um alle Änderungen im aktuellen Terminal zu aktivieren, muss die bashrc ausgeführt werden. Der "." zu Beginn des Befehls ist die Kurzform von "source".

```bash
. ~/.bashrc
```

Damit die nachfolgenden Installationen in das neu angelegte Environment installieren, muss dieses aktiviert werden:

```bash
conda activate python_workshop_env
```

Für dask-mpi werden die Pakete dask, "dask[distributed]", bokeh, ipykernel und mpi4py benötigt. Für die Übungen zusätzlich die Pakete s3fs, numpy und pandas. Der nachfolgende Befehl installiert diese in das Environment:

```bash
conda install s3fs bokeh dask ipykernel numpy pandas "dask[distributed]" mpi4py
```

Die Abfrage "Proceed (\[y\]/n)?" bitte mit "y" bestätigen.

Abschließend muss noch dask-mpi installiert werden:

```bash
conda install -c conda-forge dask-mpi
```

Um das neu erstellte Environment auf die genutzten Nodes zu kopieren, sollte dieses in ein Archiv gepackt werden:

```bash
tar -zcvf ~/miniconda3/envs/python_workshop_env.gz -C ~/miniconda3/envs/ python_workshop_env/
```

In [ ]:
import os

f = open(os.path.expanduser("~/job_dask_mpi.sh"), "w")
f.write("""#!/bin/bash -l
NODES=$(scontrol show hostname | cat)
for NODE in $NODES
do
    srun -N 1 -n 1 -w $NODE --pty /bin/bash -c "mkdir /tmp/envs \
    && cp ~/miniconda3/envs/python_workshop_env.gz /tmp/envs && \
    tar -zxvf /tmp/envs/python_workshop_env.gz --directory /tmp/envs" &
done

wait

conda activate /tmp/envs/python_workshop_env

mpirun -np 9 dask-mpi --scheduler-file ~/dask-scheduler.json --interface='ib0' --local-directory='/tmp' --no-nanny""")
f.close()

In [ ]:
os.system("sinfo_t_idle")

In [ ]:
os.system("rm -f ~/dask-scheduler.json  && sbatch -p dev_multiple --nodes=2 --ntasks=9 --ntasks-per-node=5 --time=30:00 --mem=90000mb ~/job_dask_mpi.sh")

In [ ]:
os.system("squeue --start")

In [ ]:
os.system("squeue")

In [ ]:
import os, time

while not os.path.isfile(os.path.expanduser("~/dask-scheduler.json")):
    time.sleep(1)

In [ ]:
from dask.distributed import Client
client = Client(scheduler_file=os.path.expanduser('~/dask-scheduler.json'))

In [ ]:
client

## Dask Dataframe

Ein Dask Dataframe besteht aus vielen kleinen Pandas DataFrames. Dask DataFrames können auf auf die Festplatten ausgelagert werden um Probleme zu lösen die nicht in den Arbeitsspeicher passen.


<img src="https://docs.dask.org/en/stable/_images/dask-dataframe.svg" alt="Dask DataFrame"
	title="Dask DataFrame" width="300" />

https://docs.dask.org/en/stable/_images/dask-dataframe.svg

In [ ]:
from distributed import Client
client = Client('10.0.1.162:40675')

In [ ]:
client

In [ ]:
import dask.array as da
x = da.random.random((100000,100000), chunks="16 MiB")
x

In [ ]:
y = (x + x.T) - x.mean(axis=0)

In [ ]:
y.sum().compute()

## bwUniCluster Setting

Ein Virtual Environment sollte für eine gute Performance nicht im HOME-Verzeichnis eines Benutzers liegen. Das HOME-Verzeichnis liegt auf einem zentralen File-System (LUSTRE). Für jeden Zugriff ist daher Netzwerkkommunikation notwendig. Dies kann zu Verzögerungen durch Latenzen im Netzwerk führen. Um dies zu umgehen, kann auf jedem Node einzeln ein Environment im temporär verfügbaren File-System des Nodes vorgehalten werden.

```bash
python3 -m pip install s3fs bokeh dask dask_jobqueue ipykernel numpy pandas
```
